
# 1. Análise exploratória de dados da Loggi

#2. Descrição do problema

Os códigos a seguir abordarão uma análise exploratória no banco de dados de entregas da Loggi,que é  uma empresa brasileira de logística e uma das maiores transportadoras privadas do país, será mostrado os Pacotes e bibliotecas utilizados, a exploração de dados, a manipulação dos DataFrames utilizando a biblioteca pandas, a visualização de dados utilizando o seaborn e o Insight a partir da análise eploratória dos dados.

O problema trata-se de analisarmos como está a distribuição de entregas no Distrito Federal nas regiões df-0(Sobradinho), df-1(Brasília) e df-2(Taguatinga) e visualizarmos a distribuição a partir de gráficos, para assim tirarmos conclusões.

# 3. Pacotes e bibliotecas

In [ ]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
!pip3 install geopandas;
import geopandas
import matplotlib.pyplot as plt

# 4. Exploração de dados

4.1 Coleta de dados

In [2]:
data = pd.read_json('https://raw.githubusercontent.com/Pinho-11/Projeto_AnaliseDeDados/main/dataset/deliveries.json')

In [3]:
data.to_json('deliveries.json')

4.2 Wrangling da estrutura

In [ ]:
# criando um dataframe a partir do dado bruto obtido no arquivo json
deliveries_df = pd.DataFrame(data)

# desmembrando a coluna origin em duas colunas, lat e lng
hub_origin_df = pd.json_normalize(deliveries_df["origin"])

# unindo os dataframes deliveries_df e hub_origin_df
deliveries_df = pd.merge(left=deliveries_df, right=hub_origin_df,how='inner', left_index=True, right_index=True)

# excluindo a coluna 'origin'
deliveries_df.drop('origin',axis=1)

# alterando a ordem das colunas para facilitar a vizualização
deliveries_df = deliveries_df[["name", "region", "lng", "lat", "vehicle_capacity", "deliveries"]]
deliveries_df.rename(columns={"lng": "hub_lng", "lat": "hub_lat"},inplace=True)

# manipulando a coluna deliveries
deliveries_exploded_df = deliveries_df[["deliveries"]].explode("deliveries")
deliveries_normalized_df = pd.concat([pd.DataFrame(deliveries_exploded_df["deliveries"].
                                                   apply(lambda record: record["size"])).
                                      rename(columns={"deliveries": "delivery_size"}),pd.
                                      DataFrame(deliveries_exploded_df["deliveries"].
                                                apply(lambda record: record["point"]["lng"])).
                                      rename(columns={"deliveries": "delivery_lng"}),pd.
                                      DataFrame(deliveries_exploded_df["deliveries"].
                                                apply(lambda record: record["point"]["lat"])).
                                      rename(columns={"deliveries": "delivery_lat"}),], axis= 1)

# adicionando a nova coluna deliveries no deliveries_df
deliveries_df = deliveries_df.drop("deliveries", axis=1)
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_normalized_df,how='right', left_index=True, right_index=True)
deliveries_df.reset_index(inplace=True, drop=True)

# 5. Manipulação

5.1 Enriquecimento

In [5]:
# obtendo informações sobre as coordenadas geográficas
hub_df = deliveries_df[["region", "hub_lng", "hub_lat"]]
hub_df = hub_df.drop_duplicates().sort_values(by="region").reset_index(drop=True)

geolocator = Nominatim(user_agent="ebac_geocoder")
geocoder = RateLimiter(geolocator.reverse, min_delay_seconds=1)

hub_df["coordinates"] = hub_df["hub_lat"].astype(str) + ", "
hub_df["coordinates"] = hub_df["coordinates"] + hub_df["hub_lng"].astype(str)
hub_df["geodata"] = hub_df["coordinates"].apply(geocoder)

# transformando os dados da coluna geodata em um dataframe
hub_geodata_df = pd.json_normalize(hub_df["geodata"].apply(lambda data: data.raw))
hub_geodata_df = hub_geodata_df[["address.town", "address.suburb", "address.city"]]

# manipulando o dataframe hub_geodata
hub_geodata_df.rename(columns={"address.town": "hub_town","address.suburb": "hub_suburb","address.city": "hub_city"}, inplace=True)

hub_geodata_df["hub_city"] = np.where(hub_geodata_df["hub_city"].notna(),hub_geodata_df["hub_city"],hub_geodata_df["hub_town"])
hub_geodata_df["hub_suburb"] = np.where(hub_geodata_df["hub_suburb"].notna(),hub_geodata_df["hub_suburb"],hub_geodata_df["hub_city"])

hub_geodata_df = hub_geodata_df.drop("hub_town", axis=1)

# introduzindo as informações do dataframe hub_geodata no dataframe hub_df
hub_df = pd.merge(left=hub_df, right=hub_geodata_df, left_index=True, right_index=True)
hub_df = hub_df[["region", "hub_suburb", "hub_city"]]

# unindo os dados dos dataframes deliveries_df e hub_df de acordo com os dados semelhantes na coluna 'region'
deliveries_df = pd.merge(left=deliveries_df, right=hub_df, how="inner", on="region")
deliveries_df = deliveries_df[["name","region","hub_lng","hub_lat","hub_city","hub_suburb","vehicle_capacity","delivery_size","delivery_lng","delivery_lat"]]


In [6]:
# abrindo o arquivo com a latitude e longetude dos deliveries já com a geocodificação reversa
deliveries_geodata_df = pd.read_csv('https://raw.githubusercontent.com/Pinho-11/Projeto_AnaliseDeDados/main/dataset/geodata.csv')

deliveries_geodata_df.to_csv('deliveries_geodata.csv')


In [7]:
deliveries_df = pd.merge(left=deliveries_df,right=deliveries_geodata_df[["delivery_city", "delivery_suburb"]],how="inner",left_index=True,right_index=True)
deliveries_df.head()

,name,region,hub_lng,hub_lat,hub_city,hub_suburb,vehicle_capacity,delivery_size,delivery_lng,delivery_lat,delivery_city,delivery_suburb
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,Taguatinga,Taguatinga,180,9,-48.116189,-15.848929,Ceilândia,P Sul
1,cvrp-2-df-33,df-2,-48.054989,-15.838145,Taguatinga,Taguatinga,180,2,-48.118195,-15.850772,Ceilândia,P Sul
2,cvrp-2-df-33,df-2,-48.054989,-15.838145,Taguatinga,Taguatinga,180,1,-48.112483,-15.847871,Ceilândia,P Sul
3,cvrp-2-df-33,df-2,-48.054989,-15.838145,Taguatinga,Taguatinga,180,2,-48.118023,-15.846471,Ceilândia,P Sul
4,cvrp-2-df-33,df-2,-48.054989,-15.838145,Taguatinga,Taguatinga,180,7,-48.114898,-15.858055,Sol Nascente/Pôr do Sol,Sol Nascente/Pôr do Sol


5.2 Controle de qualidade

In [8]:
# analisando se há dados NaN no DataFrame deliveries_df
deliveries_df.isna().any()

name                False
region              False
hub_lng             False
hub_lat             False
hub_city            False
hub_suburb          False
vehicle_capacity    False
delivery_size       False
delivery_lng        False
delivery_lat        False
delivery_city        True
delivery_suburb      True
dtype: bool

In [9]:

prop_df = deliveries_df[["delivery_city"]].value_counts() / len(deliveries_df)
prop_df = deliveries_df[["delivery_suburb"]].value_counts() / len(deliveries_df)

# 6. Visualização

6.1 Agregação

In [10]:
# fazendo o download dos dados do mapa do Distrito Federal
!wget -q https://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc100/go_df/versao2016/shapefile/bc100_go_df_shp.zip -O distrito-federal.zip
!unzip -q distrito-federal.zip -d ./maps
!cp ./maps/LIM_Unidade_Federacao_A.shp ./distrito-federal.shp
!cp ./maps/LIM_Unidade_Federacao_A.shx ./distrito-federal.shx

In [11]:
# abrindo arquivo com o mapa do Distrito Federal
mapa = geopandas.read_file("distrito-federal.shp")
mapa = mapa.loc[[0]]

In [12]:

hub_df = deliveries_df[["region", "hub_lng", "hub_lat"]].drop_duplicates().reset_index(drop=True)
geo_hub_df = geopandas.GeoDataFrame(hub_df,geometry=geopandas.points_from_xy(hub_df["hub_lng"],hub_df["hub_lat"]))

In [13]:

geo_deliveries_df = geopandas.GeoDataFrame(deliveries_df,geometry=geopandas.points_from_xy(deliveries_df["delivery_lng"],deliveries_df["delivery_lat"]))

6.2 Vizualização

In [ ]:
# cria o plot vazio
fig, ax = plt.subplots(figsize = (50/2.54, 50/2.54))

# plot mapa do distrito federal
mapa.plot(ax=ax, alpha=0.4, color="lightgrey")

# plot das entregas
geo_deliveries_df.query("region == 'df-0'").plot(ax=ax, markersize=1, color="red", label="df-0")
geo_deliveries_df.query("region == 'df-1'").plot(ax=ax, markersize=1, color="blue", label="df-1")
geo_deliveries_df.query("region == 'df-2'").plot(ax=ax, markersize=1, color="seagreen", label="df-2")

# plot dos hubs
geo_hub_df.plot(ax=ax, markersize=30, marker="x", color="black", label="hub")

# plot da legenda
plt.title("Entregas no Distrito Federal por Região",fontdict={"fontsize": 16})
lgnd = plt.legend(prop={"size": 15})
for handle in lgnd.legendHandles:
  handle.set_sizes([50])

In [15]:
# analisando a porcentagem de entregas por região
entregas = deliveries_df[['region','hub_city']].value_counts(normalize=True).reset_index()
entregas.rename(columns={0:'region_percent'})


,region,hub_city,region_percent
0,df-1,Brasília,0.478988
1,df-2,Taguatinga,0.410783
2,df-0,Sobradinho,0.110229


#7 Insights

Portanto, podemos notar um elevado número de entregas nas regiões **df-1** e **df-2**, com a **df-1** sendo superior na quantidade e concentração de entregas. Percebe-se também que a região **df-0** apresenta uma menor concentração de entregas, o que dificulta a mobilidade e consequentemente aumenta o consumo de combustíveis.
